<a href="https://colab.research.google.com/github/01diegoporfirio/sugestao-inteligente/blob/main/sugestao_inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import google.generativeai as gemini
import numpy as np

### Insira sua chave da GOOGLE API ###

In [ ]:
GOOGLE_API_KEY = "SUA CHAVE DA API GOOGLE"
gemini.configure(api_key=GOOGLE_API_KEY)

Selecione o modelo desejado

In [ ]:
model = "models/embedding-001" # Substitua pelo nome do modelo de embedding desejado


funcoes auxilíares

In [77]:
def embed_fn(title, description, price, discount, rating, brand, category):
    """Gera embeddings para um produto com base em seus atributos."""
    combined_text = f"{description} {price} {discount} {rating} {brand} {category}"
    embedding = gemini.embed_content(
        model=model, content=combined_text, title=title, task_type="RETRIEVAL_DOCUMENT"
    )["embedding"]
    return embedding

def gerar_e_buscar_consulta(consulta, base, model):
    """Gera embeddings para a consulta e encontra o produto mais similar."""
    embedding_da_consulta = gemini.embed_content(
        model=model, content=consulta, task_type="RETRIEVAL_QUERY"
    )["embedding"]
    produtos_escalares = np.dot(np.stack(base["Embeddings"]), embedding_da_consulta)
    indice = np.argmax(produtos_escalares)
    return base.loc[indice, ["Product", "Description", "Price", "Category", "Brand"]].to_dict()

Carregamento de dados

In [78]:
url = "https://dummyjson.com/products?select=id,title,description,price,discountPercentage,rating,stock,brand,category"
response = requests.get(url)
data = response.json()
products = data['products']

pd.DataFrame(products)

,id,title,description,price,discountPercentage,rating,stock,brand,category
0,1,iPhone 9,An apple mobile which is nothing like apple,549,12.96,4.69,94,Apple,smartphones
1,2,iPhone X,"SIM-Free, Model A19211 6.5-inch Super Retina H...",899,17.94,4.44,34,Apple,smartphones
2,3,Samsung Universe 9,Samsung's new variant which goes beyond Galaxy...,1249,15.46,4.09,36,Samsung,smartphones
3,4,OPPOF19,OPPO F19 is officially announced on April 2021.,280,17.91,4.30,123,OPPO,smartphones
4,5,Huawei P30,Huawei’s re-badged P30 Pro New Edition was off...,499,10.58,4.09,32,Huawei,smartphones
5,6,MacBook Pro,MacBook Pro 2021 with mini-LED display may lau...,1749,11.02,4.57,83,Apple,laptops
6,7,Samsung Galaxy Book,Samsung Galaxy Book S (2020) Laptop With Intel...,1499,4.15,4.25,50,Samsung,laptops
7,8,Microsoft Surface Laptop 4,Style and speed. Stand out on HD video calls b...,1499,10.23,4.43,68,Microsoft Surface,laptops
8,9,Infinix INBOOK,Infinix Inbook X1 Ci3 10th 8GB 256GB 14 Win10 ...,1099,11.83,4.54,96,Infinix,laptops
9,10,HP Pavilion 15-DK1056WM,HP Pavilion 15-DK1056WM Gaming Laptop 10th Gen...,1099,6.18,4.43,89,HP Pavilion,laptops


Histórico de compras do Usuário

In [79]:
# Histórico de produtos do usuário, mude para conferir possíveis sugestões
history_products_currenty_user = [6, 7]
# Produtos correspondentes ao histórico do usuário
matched_products = []
for product in products:
    if product['id'] in history_products_currenty_user:
        product_string = ', '.join(f"{key}: {value}" for key, value in product.items())
        matched_products.append(product_string)

# String com histórico de produtos
final_string_historico_produtos = " ".join(matched_products)

products = [
    product for product in products if product['id'] not in history_products_currenty_user
]

Cria DataFrame com embeddings

In [80]:
# Criação do DataFrame
df = pd.DataFrame(products)
df.columns = ["ID", "Product", "Description", "Price", "Discount", "Rating", "Stock", "Brand", "Category"]

# Geração de embeddings para os produtos
df["Embeddings"] = df.apply(
    lambda row: embed_fn(
        row["Product"], row["Description"], row["Price"], row["Discount"], row["Rating"], row["Brand"], row["Category"]
    ),
    axis=1
)

retorna uma sugestão  de produto

In [81]:
# Geração de sugestão
sugestao = gerar_e_buscar_consulta(final_string_historico_produtos, df, model)
print(sugestao)  # Retorna JSON

{'Product': 'Microsoft Surface Laptop 4', 'Description': 'Style and speed. Stand out on HD video calls backed by Studio Mics. Capture ideas on the vibrant touchscreen.', 'Price': 1499, 'Category': 'laptops', 'Brand': 'Microsoft Surface'}
